# Load Libraries

In [10]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv
import pyet
import matplotlib.pyplot as plt

dotenv.load_dotenv()
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [11]:
data = pd.read_parquet(path="D:\Pooya\Storegs\Dropbox\IRIMO\Export\Iran_Daily_ETo_1951_2025.parquet")

In [12]:
data.columns

Index(['date', 'station_id', 'station_name', 'region_id', 'region_name', 'lat',
       'lon', 'station_elevation', 'tmax', 'tmin', 'tm', 'umax', 'umin', 'um',
       'ffm', 'sshn', 'rrr24', 'pm', 'p0m', 'ewm', 'radglo24', 'evt', 'td_m',
       'twet_m', 'tsoil_m', 'ewsm', 'Penman', 'Penman-Monteith', 'ASCE-PM',
       'FAO-56', 'Priestley-Taylor', 'Kimberly-Penman', 'Thom-Oliver',
       'Blaney-Criddle', 'Hamon', 'Romanenko', 'Linacre', 'Turc',
       'Jensen-Haise', 'Mcguinness-Bordne', 'Hargreaves', 'FAO-24', 'Abtew',
       'Makkink', 'Oudin'],
      dtype='object')

# ETo

In [13]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [14]:
monthly_data = data.groupby(
    ['region_id', 'region_name', 'station_id', 'station_name', 'year', 'month'],
    group_keys=True
).agg({
    'date': 'count',
    'lat': 'first',
    'lon': 'first',
    'station_elevation': 'first',
    'tmax': ['mean', 'count'],
    'tmin': ['mean', 'count'],
    'tm': ['mean', 'count'],
    'umax': ['mean', 'count'],
    'umin': ['mean', 'count'],
    'um': ['mean', 'count'],
    'ffm': ['mean', 'count'],
    'sshn': ['mean', 'count'],
    'rrr24': ['sum', 'count'],
    'pm': ['mean', 'count'],
    'p0m': ['mean', 'count'], 
    'ewm': ['mean', 'count'], 
    'radglo24': ['sum', 'count'],
    'evt': ['sum', 'count'],
    'td_m': ['mean', 'count'],
    'twet_m': ['mean', 'count'],
    'tsoil_m': ['mean', 'count'],
    'ewsm': ['mean', 'count'], 
    'Penman': ['sum', 'count'], 
    'Penman-Monteith': ['sum', 'count'], 
    'ASCE-PM': ['sum', 'count'],
    'FAO-56': ['sum', 'count'],
    'Priestley-Taylor': ['sum', 'count'],
    'Kimberly-Penman': ['sum', 'count'],
    'Thom-Oliver': ['sum', 'count'],
    'Blaney-Criddle': ['sum', 'count'],
    'Hamon': ['sum', 'count'],
    'Romanenko': ['sum', 'count'],
    'Linacre': ['sum', 'count'],
    'Turc': ['sum', 'count'],
    'Jensen-Haise': ['sum', 'count'],
    'Mcguinness-Bordne': ['sum', 'count'],
    'Hargreaves': ['sum', 'count'],
    'FAO-24': ['sum', 'count'],
    'Abtew': ['sum', 'count'],
    'Makkink': ['sum', 'count'],
    'Oudin': ['sum', 'count'],
})

monthly_data.columns = ['_'.join(col).strip() for col in monthly_data.columns.values]
monthly_data.columns = [col.replace('_first', '').replace('_mean', '').replace('_sum', '') for col in monthly_data.columns]
monthly_data = monthly_data.reset_index()

monthly_data = monthly_data.round(2)

for col in monthly_data.columns:
    if col.endswith('_count'):
        if col == 'date_count':
            continue
        mean_col = col.replace('_count', '')
        monthly_data.loc[monthly_data[col] == 0, mean_col] = np.nan

for col in monthly_data.columns:
    if col.endswith('_count'):
        if col == 'date_count':
            continue
        monthly_data[col] = (monthly_data[col] / monthly_data['date_count']) * 100
        monthly_data[col] = monthly_data[col].round(2)

monthly_data.columns = [col.replace('_count', '_percent') for col in monthly_data.columns]

monthly_data.drop(columns=['date_percent'], inplace=True)

monthly_data

,region_id,region_name,station_id,station_name,year,month,lat,lon,station_elevation,tmax,tmax_percent,tmin,tmin_percent,tm,tm_percent,umax,umax_percent,umin,umin_percent,um,um_percent,ffm,ffm_percent,sshn,sshn_percent,rrr24,rrr24_percent,pm,pm_percent,p0m,p0m_percent,ewm,ewm_percent,radglo24,radglo24_percent,evt,evt_percent,td_m,td_m_percent,twet_m,twet_m_percent,tsoil_m,tsoil_m_percent,ewsm,ewsm_percent,Penman,Penman_percent,Penman-Monteith,Penman-Monteith_percent,ASCE-PM,ASCE-PM_percent,FAO-56,FAO-56_percent,Priestley-Taylor,Priestley-Taylor_percent,Kimberly-Penman,Kimberly-Penman_percent,Thom-Oliver,Thom-Oliver_percent,Blaney-Criddle,Blaney-Criddle_percent,Hamon,Hamon_percent,Romanenko,Romanenko_percent,Linacre,Linacre_percent,Turc,Turc_percent,Jensen-Haise,Jensen-Haise_percent,Mcguinness-Bordne,Mcguinness-Bordne_percent,Hargreaves,Hargreaves_percent,FAO-24,FAO-24_percent,Abtew,Abtew_percent,Makkink,Makkink_percent,Oudin,Oudin_percent
0,ALKK,Alborz,18554,Zidashte Taleghan,1951,1,36.13,50.68,2255.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
1,ALKK,Alborz,18554,Zidashte Taleghan,1951,2,36.13,50.68,2255.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
2,ALKK,Alborz,18554,Zidashte Taleghan,1951,3,36.13,50.68,2255.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
3,ALKK,Alborz,18554,Zidashte Taleghan,1951,4,36.13,50.68,2255.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
4,ALKK,Alborz,18554,Zidashte Taleghan,1951,5,36.13,50.68,2255.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693376,QOQM,Qom,99440,Kahak,2025,5,34.40,50.87,1403.10,32.10,100.00,18.04,100.00,25.07,100.00,36.13,100.00,14.87,100.00,23.45,100.00,3.39,100.00,10.69,96.77,4.21,100.00,1004.42,100.00,858.08,100.00,7.22,100.00,81513.00,96.77,356.40,100.00,1.72,100.00,13.41,100.00,15.55,100.00,34.00,100.00,175.06,96.77,247.23,96.77,245.90,96.77,245.90,96.77,154.53,96.77,151.56,96.77,207.52,96.77,130.11,96.77,190.57,96.77,423.64,96.77,202.06,96.77,220.88,96.77,217.93,96.77,215.80,96.77,180.25,96.77,202.95,96.77,165.82,96.77,155.67,96.77,146.81,96.77
693377,QOQM,Qom,99440,Kahak,2025,6,34.40,50.87,1403.10,33.87,100.00,20.21,100.00,27.04,100.00,33.67,100.00,15.43,100.00,22.38,100.00,3.95,100.00,11.86,100.00,0.20,100.00,1000.70,100.00,

# Export

In [15]:
monthly_data.to_parquet('D:\Pooya\Storegs\Dropbox\IRIMO\Export\Iran_Monthly_Daily_ETo_1951_2025.parquet')
logging.info("Monthly ETo data saved to pickle and parquet files.")

INFO:root:Monthly ETo data saved to pickle and parquet files.
